In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [2]:
device = torch.device("cpu")
model = Net().to(device)

In [3]:
from torchsummary import summary
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
           Dropout-3           [-1, 64, 12, 12]               0
            Linear-4                  [-1, 128]       1,179,776
           Dropout-5                  [-1, 128]               0
            Linear-6                   [-1, 10]           1,290
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.52
Params size (MB): 4.58
Estimated Total Size (MB): 5.10
----------------------------------------------------------------


In [4]:
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [5]:
profiler = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=1, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/mnist'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)

In [6]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        profiler.step()

In [7]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
   
train_loader = torch.utils.data.DataLoader(dataset1, 32)
test_loader = torch.utils.data.DataLoader(dataset2, 1000)
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
#     test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318748


[W CPUAllocator.cpp:219] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event


Train Epoch: 1 [320/60000 (1%)]	Loss: 1.505240
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.193794
Train Epoch: 1 [960/60000 (2%)]	Loss: 0.553569
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.754131
Train Epoch: 1 [1600/60000 (3%)]	Loss: 0.656875
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.435309
Train Epoch: 1 [2240/60000 (4%)]	Loss: 0.344343
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.239507
Train Epoch: 1 [2880/60000 (5%)]	Loss: 0.290068
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.479381
Train Epoch: 1 [3520/60000 (6%)]	Loss: 0.632200
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.101524
Train Epoch: 1 [4160/60000 (7%)]	Loss: 0.368689
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.685050
Train Epoch: 1 [4800/60000 (8%)]	Loss: 0.184994
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.450657
Train Epoch: 1 [5440/60000 (9%)]	Loss: 0.260833
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.203697
Train Epoch: 1 [6080/60000 (10%)]	Loss: 0.266782
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.301771
Train Epoch: 1 [6720/60000 (11%)]	Loss: 

Train Epoch: 1 [53120/60000 (89%)]	Loss: 0.037325
Train Epoch: 1 [53440/60000 (89%)]	Loss: 0.108828
Train Epoch: 1 [53760/60000 (90%)]	Loss: 0.031267
Train Epoch: 1 [54080/60000 (90%)]	Loss: 0.063938
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.034182
Train Epoch: 1 [54720/60000 (91%)]	Loss: 0.053226
Train Epoch: 1 [55040/60000 (92%)]	Loss: 0.070565
Train Epoch: 1 [55360/60000 (92%)]	Loss: 0.013982
Train Epoch: 1 [55680/60000 (93%)]	Loss: 0.003729
Train Epoch: 1 [56000/60000 (93%)]	Loss: 0.332125
Train Epoch: 1 [56320/60000 (94%)]	Loss: 0.050691
Train Epoch: 1 [56640/60000 (94%)]	Loss: 0.051374
Train Epoch: 1 [56960/60000 (95%)]	Loss: 0.010762
Train Epoch: 1 [57280/60000 (95%)]	Loss: 0.086679
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.168751
Train Epoch: 1 [57920/60000 (97%)]	Loss: 0.010038
Train Epoch: 1 [58240/60000 (97%)]	Loss: 0.009328
Train Epoch: 1 [58560/60000 (98%)]	Loss: 0.061450
Train Epoch: 1 [58880/60000 (98%)]	Loss: 0.002078
Train Epoch: 1 [59200/60000 (99%)]	Loss: 0.003391


In [9]:
profiler.stop()

In [ ]:
!python3 -m tensorboard.main --logdir=./log

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

I1027 23:23:48.613535 13008814080 plugin.py:429] Monitor runs begin
I1027 23:23:48.615444 13008814080 plugin.py:444] Find run directory /Users/mahamarif/Documents/Cloud and ML/log/mnist
I1027 23:23:48.615829 13042393088 plugin.py:493] Load run mnist
I1027 23:23:48.627496 13042393088 loader.py:57] started all processing
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:6006/ (Press CTRL+C to quit)
I1027 23:23:50.065148 13042393088 plugin.py:497] Run mnist loaded
I1027 23:23:50.065665 13025603584 plugin.py:467] Add run mnist
W1027 23:23:51.564023 13042393088 security_validator.py:46] In 3.0, this warning will become an error:
Requires default-src for